# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [203]:
import pandas as pd
import requests
import warnings

# Suppress all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [204]:
# 1) Seu código aqui
url =       'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=SP&tipo_interessado=Estado'
inicio =        57 #Verificar se a sigla ainda permanece neste mesmo index.
fim =           59    #Verificar se a sigla ainda permanece neste mesmo index.
estado = 'RJ'
tipo_interessado = 'Estado'
def pandfyme(url_original, start, end, uf,tipo) -> str:
    """
    Função que simula uma construção de query para buscas nos Dados abertos da Secretaria do Tesouro Nacional.
    A mesma permite requests conforme UF e Tipo de Interessado.
  
    Argumentos:
        original: String com o link original.
        start: Index do ínicio da fatia de string a ser alterada.
        end: Index do final da fatia a ser alterada.
        uf: Nova sigla de Estado a ser buscada.
        tipo: Tipo de Interessado.

    Retorno:
        df: Pandas DataFrame gerado apartir dos elementos presentes em 'items'.
   
    """
    url_upd = url_original[:start] + uf + url_original[end:]

    if "tipo_interessado=" in url_upd:
        url_upd = url_upd.split("&tipo_interessado=")[0]
        url_upd += "&tipo_interessado=" + tipo
    print(f'Requisitando da URL: \n{url_upd}\n')

    r = requests.get(url_upd)
    if r.status_code == 200:
        print(f'Êxito: {r.status_code}\nJSON recebido!')
        data_json = r.json()
            
    df = pd.DataFrame(data_json['items'])
    print('JSON transformado em DataFrame do Pandas')
    

    return df
    

data = pandfyme(url,inicio,fim,estado,tipo_interessado)

data.head()

Requisitando da URL: 
http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado

Êxito: 200
JSON recebido!
JSON transformado em DataFrame do Pandas


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13483,Estado,Rio de Janeiro,33,RJ,None,Deferido,19405.000010/2004-52,2006-02-15T02:00:00Z,Operação contratual interna,Renegociação de dívidas,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,8.475263e+08,1,0,01/03/2006
1,13629,Estado,Rio de Janeiro,33,RJ,None,Deferido,19405.000040/2002-05,2002-06-20T03:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,4.352755e+07,1,0,27/06/2002
2,9102,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000903/2009-80,2009-07-29T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco do Brasil S/A,Real,6.110800e+07,1,0,06/08/2009
3,5923,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000234/2012-41,2012-04-16T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,2.000000e+08,1,0,04/10/2012
4,21660,Estado,Rio de Janeiro,33,RJ,00000.000000/2026-68,Arquivado a pedido,17944.000245/2016-55,2016-03-11T03:00:00Z,Operação contratual interna (com garantia da U...,Fortalecimento Institucional,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1.000000e+09,0,0,16/05/2016


In [205]:
df_status=pd.DataFrame(data['status'].value_counts()).reset_index()
df_status.columns=['status', 'frequencia']
df_status
print(f'Os três valores de status mais frequentes são:\n1º {df_status["status"][0]}\n2º {df_status["status"][1]}\n3º {df_status["status"][2]}\n')
print("Tabela de Frequencias")
df_status

Os três valores de status mais frequentes são:
1º Encaminhado à PGFN com manifestação técnica favorável
2º Deferido
3º Arquivado a pedido

Tabela de Frequencias


,status,frequencia
0,Encaminhado à PGFN com manifestação técnica fa...,34
1,Deferido,14
2,Arquivado a pedido,3
3,Arquivado por decurso de prazo,1


In [206]:
df_ano = data[['data_status','status']]
anos = pd.DataFrame(df_ano['data_status'].value_counts()).reset_index()
anos.columns = ['ano', 'frequencia']
anos['ano'] = anos['ano'].apply(str).apply(lambda x: x[-4:])
anos



,ano,frequencia
0,2002,4
1,2011,4
2,2012,2
3,2012,2
4,2014,2
5,2013,2
6,2006,1
7,2012,1
8,2009,1
9,2012,1


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [207]:
#Gerando Pandas DataFrame conforme inputs fornecidos no enunciado
url =       'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=SP&tipo_interessado=Estado'
inicio =        57 #Verificar se a sigla ainda permanece neste mesmo index.
fim =           59    #Verificar se a sigla ainda permanece neste mesmo index.
estado = 'MG'
tipo_interessado = 'Estado'
df_mg = pandfyme(url,inicio,fim,estado,tipo_interessado)


Requisitando da URL: 
http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado

Êxito: 200
JSON recebido!
JSON transformado em DataFrame do Pandas


In [208]:
print(f'O número de solicitações com o status "Arquivado por decurso de prazo" é: {df_mg["status"].value_counts()["Arquivado por decurso de prazo"]}')

O número de solicitações com o status "Arquivado por decurso de prazo" é: 1


In [209]:
#Gerando Pandas Dataframe para o estado da Bahia com as solicitações dos municípios.
url =       'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=SP&tipo_interessado=Estado'
inicio =        57 #Verificar se a sigla ainda permanece neste mesmo index.
fim =           59    #Verificar se a sigla ainda permanece neste mesmo index.
estado = 'BA'
tipo_interessado = 'Município'
df_ba = pandfyme(url,inicio,fim,estado,tipo_interessado)



Requisitando da URL: 
http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Município

Êxito: 200
JSON recebido!
JSON transformado em DataFrame do Pandas


In [210]:
muni_defe = df_ba[df_ba['status'] == 'Deferido']
df_muni_defe = muni_defe['interessado'].value_counts().reset_index()
df_muni_defe.columns = [['interessado', 'qt_deferidas']]
interessado = df_muni_defe['interessado'].iloc[0][0]
qt_deferidas = df_muni_defe['qt_deferidas'].iloc[0][0]
print(f"O município do Estado da Bahia com o maior número de solicitações deferidas é: {interessado} com {qt_deferidas} solicitações deferidas.")


O município do Estado da Bahia com o maior número de solicitações deferidas é: Luís Eduardo Magalhães com 16 solicitações deferidas.
